# CrewAI Coder Agent - Control Center

This notebook provides an interactive interface to control and monitor the CrewAI coder agent.

## Features
- Configure and run the agent
- Monitor execution in real-time
- View logs and metrics
- Customize prompts and settings
- Inspect generated files

## Setup and Imports

In [ ]:
# Imports
import sys
import time
from datetime import datetime
from pathlib import Path

from helpers.logger import SessionLogger
from helpers.config_loader import load_config
from core.agent_factory import create_coder_crew
from core.context_loader import load_prompt_template

print("✓ All imports successful")

## Load Configuration

In [ ]:
# Load config
config = load_config()

print("Configuration:")
print(f"  LLM Model: {config.get('llm.model')}")
print(f"  Base URL: {config.get('llm.base_url')}")
print(f"  Temperature: {config.get('llm.temperature')}")
print(f"  Max Tokens: {config.get('llm.max_tokens')}")
print(f"  Agent Role: {config.get('agent.role')}")
print(f"  Max Iterations: {config.get('agent.max_iterations')}")

## Initialize Session Logger

In [ ]:
# Create session logger
session_id = f"notebook_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
logger = SessionLogger(session_id=session_id)

logger.print_success(f"Session started: {session_id}")
logger.print_info(f"Logs will be saved to: logs/{session_id}.log")

## View Default Prompt

In [ ]:
# Load and display the default prompt
default_prompt = load_prompt_template('build-website')
print("Default Task Prompt:")
print("=" * 80)
print(default_prompt[:1000])  # Show first 1000 characters
print("...")
print(f"\nTotal length: {len(default_prompt)} characters")

## Option 1: Run with Default Prompt

In [ ]:
# Create and run crew with default settings
logger.print_section("Creating CrewAI Crew")

crew = create_coder_crew(verbose=True)

logger.print_success("Crew created successfully")
logger.print_info(f"Agents: {len(crew.agents)}")
logger.print_info(f"Tasks: {len(crew.tasks)}")

In [ ]:
# Execute the crew
logger.print_section("Executing Task")
logger.print_info("Building personal website...")
logger.print_info("This may take several minutes.\n")

start_time = time.time()

try:
    result = crew.kickoff()
    
    execution_time = time.time() - start_time
    
    logger.print_section("Execution Complete")
    logger.print_success(f"Completed in {execution_time:.2f} seconds")
    
    print("\nResult:")
    print("=" * 80)
    print(result)
    
except Exception as e:
    logger.print_error(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

## Option 2: Run with Custom Prompt

In [ ]:
# Define custom prompt
custom_prompt = """
Create a minimalist personal portfolio website with the following requirements:

Technical Stack:
- React 18 with functional components
- Tailwind CSS for styling
- Vite as build tool

Sections:
1. Hero section with name and title
2. About section
3. Skills grid
4. Project showcase (3 projects minimum)
5. Contact form

Requirements:
- Fully responsive design
- Dark mode toggle
- Smooth scrolling navigation
- Modern, clean aesthetic
- All code must be well-documented
- Include package.json and README

Output everything to ./output directory.
"""

print("Custom Prompt:")
print(custom_prompt)

In [ ]:
# Create crew with custom prompt
custom_crew = create_coder_crew(custom_prompt=custom_prompt, verbose=True)

logger.print_success("Custom crew created")

In [ ]:
# Execute custom crew
start_time = time.time()

try:
    result = custom_crew.kickoff()
    
    execution_time = time.time() - start_time
    logger.print_success(f"Custom task completed in {execution_time:.2f} seconds")
    
    print(result)
    
except Exception as e:
    logger.print_error(f"Error: {str(e)}")
    import traceback
    traceback.print_exc()

## Inspect Generated Files

In [ ]:
# List all files in output directory
import os

output_dir = Path("./output")

if output_dir.exists():
    print("Generated Files:")
    print("=" * 80)
    
    for root, dirs, files in os.walk(output_dir):
        level = root.replace(str(output_dir), '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            file_path = Path(root) / file
            size = file_path.stat().st_size
            print(f"{subindent}{file} ({size:,} bytes)")
else:
    print("Output directory not found or empty")

## View Specific Files

In [ ]:
# Read and display a specific file
file_to_view = "output/package.json"  # Change this to view different files

try:
    with open(file_to_view, 'r') as f:
        content = f.read()
    
    print(f"File: {file_to_view}")
    print("=" * 80)
    print(content)
    
except FileNotFoundError:
    print(f"File not found: {file_to_view}")
except Exception as e:
    print(f"Error reading file: {e}")

## View Session Logs

In [ ]:
# View session metrics
import json

json_log = Path(f"logs/{session_id}.json")

if json_log.exists():
    with open(json_log, 'r') as f:
        session_data = json.load(f)
    
    print("Session Metrics:")
    print("=" * 80)
    print(json.dumps(session_data.get('metrics', {}), indent=2))
    
    print("\nRecent Events:")
    print("=" * 80)
    events = session_data.get('events', [])
    for event in events[-5:]:  # Show last 5 events
        print(f"\n[{event['timestamp']}] {event['type']}")
        if 'data' in event:
            print(json.dumps(event['data'], indent=2)[:200])
else:
    print("Session log not found yet")

## End Session

In [ ]:
# End session and save final logs
logger.end_session()

print("\n✓ Session ended. Logs saved.")

## Quick Reference

### Common Tasks

1. **Run with default settings**: Execute the first "Option 1" cells
2. **Run with custom prompt**: Modify and execute "Option 2" cells
3. **Check output**: Run "Inspect Generated Files" cell
4. **View logs**: Run "View Session Logs" cell

### Configuration Changes

Edit `config.yaml` to change:
- LLM settings (model, temperature, etc.)
- Agent behavior (role, goal, max iterations)
- Output directory
- Logging settings

### Context Files

Modify these to change agent behavior:
- `context/no-goes.md` - Safety rules
- `context/guidelines/coding-standards.md` - Code standards
- `context/skills/` - Knowledge base
- `prompts/build-website.md` - Default task prompt

### Output Directory

All generated files go to `./output/`
This directory is gitignored but tracked for changes.

### Logs Directory

Session logs are saved to `./logs/`
- `session_*.log` - Human-readable log
- `session_*.json` - Structured data with metrics
